#문서 색인, 검색 코드 만들어보기

이번 과제에 대해서 문서를 크롤링 하고 가장 비슷한 내용을 최상단으로 보내는 코드를 만드는 활동을 진행하였다.


엑셀, txt, 한글, pdf를 색인하고 검색 결과 도출을 목표로 진행하기 시작했음.


완료 된 것 : 엑셀, txt, pdf / 한글(안됨)

In [83]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [82]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install chromadb

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy/main Sources [2,197 kB]
Fetched 2,215 kB in 8s (267 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... D

#크롤링
게임메카 프리뷰와 리뷰의 뉴스를 크롤링 하여서 기사 제목을 문서 제목으로 정하고 내용을 .txt화 시켜서 폴더 내로 보내기

colab 이 문제인지 코드가 문제인지를 모르겠지만 colab으로 실행시키면 광고에 막히는 문제점이 있음..

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import time
import re
import os

options = webdriver.ChromeOptions()
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36')
options.add_argument("disable-images")
options.add_argument("--disable-default-image-capturing")

driver = webdriver.Chrome(options=options)

url = "https://www.gamemeca.com/review.php"
driver.get(url)
driver.maximize_window()
for i in range(1,6):

  for j in range(1,5):
    text_list = []


    driver.implicitly_wait(60)
    review =  driver.find_element(By.XPATH, "//*[@id='content']/div[5]/ul[" + str(i) + "]/li[" + str(j) + "]"  )
    review.click()
    driver.implicitly_wait(60)

    title = driver.find_elements(By.CSS_SELECTOR, 'div.article')
    print("진행중")
    for x in title:
      text_list.append(x.text)
    text = " ".join(text_list)

    pattern = r'\([^)]*\)'  # () 제거
    text = re.sub(pattern=pattern, repl='', string=text)

    filename = driver.find_element(By.CLASS_NAME, 'article-title').text
    filename = re.sub(r'[\/:*?"<>|\n]', '', string=filename)
    file_path = os.path.join('file_text', filename + '.txt')

    with open(file_path,'w', encoding="utf-8") as flie:
        flie.write(text)

    driver.back()


In [84]:
import chromadb
chroma_client = chromadb.Client()
print(chroma_client)
print(chromadb.__version__)

0.4.13


In [ ]:
!pip install -q langchain unstructured openai tiktoken chromadb

In [ ]:
!pip install sentence-transformers

In [85]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader


from langchain.llms import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings

In [88]:
!pip install PyMuPDF
!pip install pyhwp

엑셀, txt, pdf 파일 인덱스

이후 hwp 추가 예정

In [87]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders.pdf import PyMuPDFLoader
from langchain.document_loaders.xml import UnstructuredXMLLoader
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.document_loaders import TextLoader

# 파일 종류에 따라서 Loader 지정
loaders = {
    '.pdf': PyMuPDFLoader,
    '.xml': UnstructuredXMLLoader,
    '.csv': CSVLoader,
    '.txt': TextLoader
}

# Loader 함수 지정
def create_directory_loader(file_type, directory_path):
    return DirectoryLoader(
        path=directory_path,
        glob=f"**/*{file_type}",
        loader_cls=loaders[file_type],
    )

# Create DirectoryLoader instances for each file type
pdf_loader = create_directory_loader('.pdf', '/content/drive/MyDrive/file_text')
xml_loader = create_directory_loader('.xml', '/content/drive/MyDrive/file_text')
csv_loader = create_directory_loader('.csv', '/content/drive/MyDrive/file_text')
txt_loader = create_directory_loader('.txt', '/content/drive/MyDrive/file_text')

# Load the files
pdf_documents = pdf_loader.load()
xml_documents = xml_loader.load()
csv_documents = csv_loader.load()
txt_documents = txt_loader.load()

print(len(txt_documents))
print(len(pdf_documents))
txt_documents = txt_documents + pdf_documents + hwpx_docments
print(len(txt_documents))

ModuleNotFoundError: ignored

text 분할

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
txt_splitter = text_splitter.split_documents(txt_documents)

print(len(txt_splitter))

90


In [ ]:
embedding = HuggingFaceEmbeddings()

db = Chroma.from_documents(
    txt_splitter,
    embedding=embedding,
    persist_directory='flie_text_append')

db.persist()

In [ ]:
query = "소울라이크 힐링 불 게임을 즐기는 방법"
docs = db.similarity_search(query, metric="cosine")

검색 결과를 0~5 까지 출력

(colab 의 택스트 한계로 인해서 4까지 밖에 나타나지 않았음.)

In [ ]:
docs[:5]

[Document(page_content='적의 타입은 크게 인간, 인형, 괴물로 분류된다. 인간형 적은 우리가 흔히 아는 움직임을 가지고 있고, 대부분 무기를 장착하고 있다. 인형형 적은 관절 구동이 자유로워 공격 범위가 넓고 강한 대신 모션이 크고 정직하다. 괴물형 적은 변화무쌍한 촉수 공격과 공격 시 부식 디버프를 가지고 있되, 초반 인식 모션이 느려 선공이나 경직을 주기가 쉽다.\n\n▲ 인간형 적의 또다른 특징으로는 비교적 가벼워 퍼펫 스트링으로 끌어올 수 있다는 점이다 \n\n▲ 인형형 적들은 항상 예상치 못한 움직임을 선보여 진행 중 큰 고통을 선사했다 \n\n앞서 등장한 경고문구와 속성의 특징, 그리고 적의 분류를 보면, 인형형 적을 처치하는 것에는 전격 상태이상을 입힐 수 있는 리전 암, 혹은 해당 속성을 가진 소모품이 도움이 된다는 것을 자연스레 알 수 있다. 아울러 범위가 넓어 회피는 힘들지만 모션이 크고 정직해 퍼펙트 가드가 쉽다는 결론도 내릴 수 있다. 게임 내에는 이런 판단을 도울 수 있는 상황이나 코멘트가 다수 배치돼 있어, 맵이나 주변 환경을 잘 파악하기만 해도 게임의 이해도를 높일 수 있다.\n\n주인공은 이제 체간 깎는 인형이 아닙니다\n\nP의 거짓 데모판에서 가장 많이 나온 문제점은 회피의 불필요함이었다. 회피의 모션이 부자연스러운 점도 있지만, 인형형 적이 주로 등장해 그 필요성이 크게 부각되지 않았다. 특히 데모 단계에서는 퍼펙트 가드만으로도 손쉽게 파훼할 수 있는 보스만 등장했기에 회피는 사실상 유명무실하다는 평을 받았다. 일각에서는 “생긴 건 블러드본에 플레이는 세키로”라는 평도 나왔다.\n\n하지만 정식 출시판에서는 이 밸런스가 크게 조정됐다. 회피가 더욱 자연스럽고, 모션도 다양해졌기 때문이다. 회피 후 공격 연계도 이전마냥 어색한 느낌 없이 매끄럽게 이어진다. 아울러 이후 등장하는 적의 타입이 각각 뚜렷한 특성을 가진 만큼 퍼펙트 가드만을 고수하기에는 다소 난이도가 높아졌다.\n\n▲ 데모판에서 퍼펙트 가드만으로

#첫 시도의 문서 색인, 검색
문서를 읽고 id를 구별해서 찾고 검색 결과를 비슷한 것을 점수를 매겨서 역순으로 출력하는 것을 시도 하였음.


폴더 인식이 되지 않아서 다른 방법을 모색.

In [ ]:
import chromadb
import os

# chroma DB 연결
client = chromadb.Client()
db.persist()
# 문서 색인
def index_document(document):
    # 폴더를 열고 문서를 읽음
    for root, _, files in os.walk(document["/file_text"]):
        for file in files:
            # 문서 내용을 색인
            index = db.create_index("documents", "text", open(os.path.join(root, file), "r").read())
            # 문서 ID를 색인
            index.add_document(document["id"])


# 문서 검색
def search_documents(query):
    # 검색어를 색인
    index = db.get_index("documents", "text")
    # 검색 결과를 정렬
    results = index.search(query, sort_by="score", reverse=True)
    # 검색 결과를 반환
    return results

index_document({"path": "/content/file_text"})

print(search_documents("힐링"))

KeyError: ignored